# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [3]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount=''
drive_root = mount + ""

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [4]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

Archivos en el directorio: 
['.git', '.gitignore', 'intro_gym_VZ_colab.ipynb', 'Proyecto_práctico_colab.ipynb', 'Proyecto_práctico_local.ipynb', 'README.md', 'rtest.py']


---
### 1.4. Instalar librerías necesarias
(descomenta si es la primera vez que corres el codigo)

In [5]:
# if IN_COLAB:
#   %pip install gym==0.17.3
#   %pip install git+https://github.com/Kojoley/atari-py.git
#   %pip install keras-rl2==1.0.5
#   %pip install tensorflow==2.8
# else:
#   %pip install gym==0.17.3
#   %pip install git+https://github.com/Kojoley/atari-py.git
#   %pip install pyglet==1.5.0
#   %pip install h5py==3.1.0
#   %pip install Pillow==9.5.0
#   %pip install keras-rl2==1.0.5
#   %pip install Keras==2.2.4
#   %pip install tensorflow==2.5.3
#   %pip install torch==2.0.1
#   %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [6]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, concatenate, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

c:\Users\javie\anaconda3\envs\gym\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\javie\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\javie\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


#### Configuración base

In [7]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [8]:
print("Numero de acciones disponibles:" + str(nb_actions))

Numero de acciones disponibles:6


In [10]:
print("Formato de las observaciones:")
env.observation_space

Formato de las observaciones:


Box(0, 255, (210, 160, 3), uint8)

Un espacio de soluciones de dimensiones (210,160,3) tiene un tamaño inmanejable. Se puede reducir ese espacio a un tamaño de 84x84 con una window length 4.

In [11]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

##1. Implementación de la red neuronal

Se puede hacer:


1.   DQN
2.   Doble DQN
3.   Duel DQN

Y luego también variar el discount factor, cambios en epsilon, tamaño de memoria, uso de epsilon greedy vs. boltzman y escribir como varía el resultado. Cambiar el número de episodios es un poco gilipollez, dara mejores resultados por más train. Usamos un número pequeño de episodios. el mínimo para que llegue a 20 puntos y a partir de ahí vamos probando. También podríamos cambiar el input de la red neuronal.

## VER EN CONJUNTO





In [12]:
def construct_q_network(num_actions,dim=(84,84,4), loss='mse', optimizer=Adam(lr=0.00001)):
    # Uses the network architecture found in DeepMind paper
    model = Sequential()
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), input_shape=dim))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(num_actions))
    model.compile(loss=loss, optimizer=optimizer)
    print("Successfully constructed networks.")
    return model

c:\Users\javie\anaconda3\envs\gym\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [13]:
def construct_double_q_network(input_shape, num_actions, loss='mse', optimizer=Adam(lr=0.000001)):
        # arquitectura de DeepMind paper

      input_layer = Input(shape=input_shape)
      conv1 = Convolution2D(32, (8, 8), strides=(4, 4), activation='relu')(input_layer)
      conv2 = Convolution2D(64, (4, 4), strides=(2, 2), activation='relu')(conv1)
      conv3 = Convolution2D(64, (3, 3), activation='relu')(conv2)
      flatten = Flatten()(conv3)

      fc1 = Dense(512)(flatten)
      advantage = Dense(num_actions)(fc1)

      fc2 = Dense(512)(flatten)
      value = Dense(1)(fc2)

      policy = concatenate([advantage, value], axis=-1)
      policy = Lambda(lambda x: x[:, :-1] - K.mean(x[:, :-1], axis=1, keepdims=True) + x[:, -1])(policy)

      model = Model(inputs=input_layer, outputs=policy)
      model.compile(loss=loss, optimizer=optimizer)

      target_model = Model(inputs=input_layer, outputs=policy)
      target_model.compile(loss=loss, optimizer=optimizer)

      print("Successfully constructed networks.")

      return model, target_model
      # El target puede ser parametro del agents

In [14]:
def construct_duel_q_network(input_shape, num_actions, loss='mse', optimizer=Adam(lr=0.000001)):
    input_layer = Input(shape=input_shape)
    conv1 = Convolution2D(32, (8, 8), strides=(4, 4), activation='relu')(input_layer)
    conv2 = Convolution2D(64, (4, 4), strides=(2, 2), activation='relu')(conv1)
    conv3 = Convolution2D(64, (3, 3), activation='relu')(conv2)
    flatten = Flatten()(conv3)

    fc1 = Dense(512)(flatten)
    advantage = Dense(num_actions)(fc1)

    fc2 = Dense(512)(flatten)
    value = Dense(1)(fc2)

    policy = concatenate([advantage, value], axis=-1)
    policy = Lambda(lambda x: x[:, :-1] - K.mean(x[:, :-1], axis=1, keepdims=True) + x[:, -1])(policy)

    model = Model(inputs=input_layer, outputs=policy)
    model.compile(loss=loss, optimizer=optimizer)

    target_model = Model(inputs=input_layer, outputs=policy)
    target_model.compile(loss=loss, optimizer=optimizer)

    print("Successfully constructed networks.")

    return model, target_model


##2. Implementación de la solución DQN

In [15]:
memory = SequentialMemory(limit=50000, window_length=1)

In [16]:
policy = BoltzmannQPolicy() # LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy

Ver aquí parámetros del https://www.tensorflow.org/agents/api_docs/python/tf_agents/agents/DqnAgent

In [28]:
#model=construct_duel_q_network(input_shape=(84,84,4), num_actions = 6,  loss='mse', optimizer=Adam(lr=0.00001))

model=construct_q_network(6,dim=(84,84,4), loss='mse', optimizer=Adam(lr=0.00001))
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory,
               nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy) #target model te congela durante 100 steps lo que se usa para el "ground truth"
               #el warmup hace una policy aleatoria para ir descubriendo estados y viendo como furrula aprovechando que hay exp replay

dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

Successfully constructed networks.


In [29]:
# Training part
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

dqn.fit(env, callbacks=callbacks, nb_steps=1750000, log_interval=10000, visualize=False)

dqn.save_weights(weights_filename, overwrite=True)

Training for 1750000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected conv2d_24_input to have 4 dimensions, but got array with shape (1, 1, 210, 160, 3)

In [ ]:
# Testing part to calculate the mean reward
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---